<h1><b>Recuperação de Informação na Web 2020/2</b></h1>

<h4>Construção de um Coletor de Propósito Geral</h4>

Integrantes:    <ul><li>Antonio</li>
                <li>Mariana Bulgarelli Alves dos Santos</li>
                <li>Nathã Paulino</li></ul>
-----------------------------------------------------------------------------------------------------------------------


O coletor (crawler) busca realizar a coleta de dados de fontes diversas. Ele é composto de três partes principais: Downloader, Scheduler e Storage. O Downloader baixa cada uma das páginas enquanto o Scheduler mantém uma relação de páginas a serem requisitadas. No storage são armazenados os dados para possíveis  busca e indexação além de fornecer Metadados ao escalonador sobre as páginas baixadas.

Basicamente, o Crawler realiza o dowload de um grupo de sementes que são parseadas e visitadas para a coleta de novos links. Os links para páginas que não tiveram seu download realizado são armazenadas em uma fila para serem posteriormente coletadas.O crawler seleciona uma nova página para ser coletada e o processo segue até atingir o nível 6.


Profundidade de coleta: 6

Número máximo de páginas: 1.000 páginas


A arquitetura do crawler é apresentada a seguir:

<figure style="text-align:center">
    <img src="imgs/arquitetura.png">
    
    Fonte: [1]
</figure>

O coletor de URL's elaborado é composto de três classes principais, a saber:
<ul><li>Domain</li>
<li>Page_fetcher: threads que requisitam as URLs obtidas com o escalonador</li>
<li>Scheduler: armazena as filas de URLs a serem requisitadas</li></ul>


As URL's de interesse são armazenadas em uma fila, que no caso é um dicionário.


<p>

A Classe <b>Domain</b> possui `time_last_access`, nam_domain (nome do domínio) e `int_time_limit_seconds` como atributos e armazena informações úteis para o momento da coleta, sendo que `int_time_limit_seconds` e `nam_domain` são passados como parâmetro.
    
O atributo `int_time_limit_seconds` indica de quanto em quanto tempo as requisições podem ser realizadas. O atributo `time_last_access` registra o momento do último aceeso à pagina.

Assim, ao tentar realizar uma requisição, primeiramente verifica-se se o servidor está acessível por meio da função `is_accessible` que verifica quando foi o último acesso comparando o retorno da função `time_since_last_access(self)` com `int_time_limit_seconds`. A `time_since_last_access` retorna um objeto TimeDelta com a diferença da data atual e a data do último acesso (`time_last_access`). Se o retorno for menor que `int_time_limit_seconds`, a função `is_accessible` retorna True. Assim, ao efetivar o acesso à página, a função accessed_now deve ser chamada e o atributo `time_last_access` modificado para o momento do acesso.

Além dessas funções, a classe Domain contém outras quatro funções que irão auxiliar na manipulação das urls. Como utilizamos um dicionário ordenado onde a key é um objeto da classe Domain e o value é uma lista de urls, para viabilizar a busca de modo direto pelo domínio utilizamos as funções `__hash__` (retorna o valor hash de um objeto, se houver - associa um valor de retorno para o objeto) e `__eq__`.

Os métodos `__str__` e `__repr__` retornam uma string que representa o objeto, no caso, `nam_domain`.
O tempo entre as requisições foi definido como 20 segundos.</p>


----------------------------------------------------------------------------------------------------------------------

<p> A Classe <b>Sheduler</b> trata-se do escalonador, que é responsável por armazenar e gerenciar a fila de urls. Confome explicado anteriormente, nossa fila de urls é na verdade um dicionário ordenado onde a key é um objeto da classe Domain (servidor) e value consiste em uma lista de tuplas de url e profundidade como objetos da classe parseResult (urlparse divide uma string de URL em seus componentes ou na combinação de componentes de URL em uma string de URL). Na coleta a profundidade foi limitada.
    
A Classe <b>Scheduler</b> possui como atributos `str_usr_agent` (nome do coletor - passado por parâmetro), `int_page_limit` (número de páginas a serem coletadas - passado por parâmetro), `int_depth_limit` (profundidade máxima a ser coletada - passado por parâmetro), `int_page_count` (quantidade de páginas já coletadas), `dic_url_per_domain` (OrderedDict() com a fila de URLs por domínio), `set_discovered_urls` (conjunto de URLs extraídas em algum HTML e já adicionadas na fila), `dic_robots_per_domain` (dicionário que contém o objeto com as regras do robots.txt para cada domínio) e `arr_urls_seeds` (array contendo urls, sendo que pra casa uma é criado um objeto urlparse e adicionado com profundidade de coleta 0 com `add_new_page`).
    
A função `can_add_page` verifica se a página não foi descoberta ainda e se a profundidade dela é menor que a profundidade limite e se as condicoes sao verdadeiras, retorna True. A função `get_next_url` obtem uma nova URL por meio da fila. Para isso, pega as keys do dicionario de urls e itera sobre elas verificando se o dominio esta acessivel (`is_accessible`). Se sim, é marcado como acessado, a url é removida da fila e fica em espera pelo tempo entre requisições.
    
A função `count_fetched_page` conta o número de paginas já coletadas e a `has_finished_crawl` verifica se a coleta foi finalizada, retornando true em caso verdadeiro.

A função `add_new_page` recebe como parâmetros um objeto da classe ParseResult com a URL a ser adicionada e a profundidade na qual foi coletada essa URL. Esta função adiciona uma nova página caso`can_add_page` retorne verdadeiro. Inicialmente é criado um objeto Domain com o netloc do objeto da classe ParseResult com a URL e com o tempo entre requisições para em seguida verificar se a página pode ser adicionada, retornando false se não puder. Assim, a url não é adicionada e o retorno é False. Caso contrário, adiciona-se o objeto como key do dicionario de urls por dominio com a profundidade de coleta como value e indica-se que essa página já foi descoberta.

Fizemos duas tratativas para as páginas que não podem ser coletadas. Uma avaliando o robots.txt e outra verificando o robots meta tag, que fornece instruções de coleta dentro do header do html da página. Nesse header, dentro da tag meta com name="robots", quando o content é igual a "noindex" or "none" a página não pode ser coletada. Quando consta como "nofollow", a página pode ser coletada, mas nenhum link em seu interior pode ser seguido.
    
Já a função `can_fetch_page` informa se uma url pode ou não ser coletada com base nas informações fornecidas pelo robots.txt. Se não for possível a coleta é lançada uma exceção.</p>

<p>A Classe <b>PageFetcher</b> é responsável por gerenciar as threads que rodam em paralelo. Uma thread é fluxo de controle de instruções e proporcionam capacidade de resposta mais ágil que sua não utilização. A possui como atributos `obj_scheduler` (passado como parâmetro) e `crawlerName`. A função `run` contém um loop que roda enquanto houverem páginas a serem coletadas (enquanto a função `has_finished_crawl` retornar False) coletando todos os conjuntos de tuplas da crawl (retorno da `discovery_links`). A cada execução, a `run` chama a função `crawl_new_url` que coleta uma nova url do escalonador (`get_next_url`) e verifica se pode ser coletada avaliando o robots.txt (`can_fetch_page`). Se o retorno da `crawl_new_url` for diferente de vazio, acrescenta-se um ao número de páginas coletadas e, então, para cada tupla do conjunto, verificar se é possível adicionar. Se é, adiciona no contador.
    
A função `request_url` realiza a requisição da url e retorna o conteúdo em binário da url. Foi necessário tratar as variações de retorno das urls. A função `discover_links` recolhe os links de todas as páginas. </p>

-----------------------------------------------------------------------------------------------------------------

<h3>URLs sementes utilizadas</h3>
<p>Foram utilizadas as seguintes urls como seed:
    <ui><li>http://www.crea-mg.org.br</li>
    <li>https://www.cefetmg.br</li>
   <li>https://youtube.com</li></ui>
</p>

<h3>Principais desafios, decisões e arquitetura utilizada</h3>
<p> Durante o desenvolvimento do código encontramos algumas dificuldades no que tange a tratativa de erros. Neste caso optamos por utilizar tratamento de exceção da própria linguagem. Para os casos mais simples, como o retorno de None, tratamos localmente com estruturas condicionais.

A arquitetura utilizada foi anteriormente descrita neste relatório. Vide Figura 1.</p>

<h3>O impacto na velocidade de coleta (quantidade de páginas por segundo) ao aumentar o número de threads 10 a 100, de 20 em 20</h3>

<h5>Observações: Número de páginas coletadas = 1000 -- A lista de urls coletadas encontra-se em Coding Dojo - Crawler</h5>
Através da análise dos resultados das coletas, percebe-se que à medida que aumentamos o número de threads, o número de páginas coletadas a cada segundo aumenta ligeiramente. O resultado pode ser observado na tabela a seguir:


<h4>Bibliografia</h4>
[1] Dalip, D. H. Recuperação de Informação - Coletores na Web: arquitetura e política de escolha e "revisita" de páginas. Centro Federal de Educação Tecnológica de Minas Gerais, 2020.

In [ ]:
Bibliotecas utilizadas:
from datetime import datetime
from collections import OrderedDict

from urllib import robotparser
from util.threads import synchronized
from collections import OrderedDict
from crawler.domain import Domain
from urllib.parse import urlparse, urlunparse
import time 
import urllib.robotparser
pandas

from bs4 import BeautifulSoup
from threading import Thread
import requests
from urllib.parse import urlparse, urlunparse, urljoin

<h3>Link para a página descrevendo o coletor criado</h3>



In [ ]:
https://nathaclmpaulino.github.io/crawlerRI/infoCrawlerBot/